In [1]:
import warnings
warnings.filterwarnings('ignore')

import ast
import math
import pandas as pd
import glob
import numpy as np
# import cv2

# from simplification.cutil import simplify_coords

import matplotlib.pyplot as plt
import matplotlib.style as style

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
import os
import datetime as dt
from tqdm import tqdm
import pandas as pd
import numpy as np
from glob import glob
import re
import ast
# import cv2
import csv
import time
import ast
import urllib
from PIL import Image, ImageDraw
from tqdm import tqdm
from dask import bag, threaded
import matplotlib
import matplotlib.pyplot as pltc
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
from dask import bag, threaded
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.applications import MobileNet
from keras.models import Sequential,Model  #Input

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# from keras.layers.normalization import BatchNormalization
# from keras.layers.advanced_activations import LeakyReLU
import os

Base_Size = 256
num_csv = 100
num_class = 340
size = 64
steps = 1
batchsize = 256
epochs = 100




2024-07-08 17:08:46.317527: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 17:08:46.446349: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 17:08:47.003208: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 17:08:51.000845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
# compress train_simplified csv files
import pandas as pd
import os
from time import time


def compress_csv_files(from_folder,save_to_folder):
    i=0
    for file in os.listdir(from_folder):
        print("ih")
        if file[-4:] == ".csv":
            print(i)
            print("-"*40)
            print(f"Converting File: {file}")
            start = time()
            df = pd.read_csv(f"{from_folder}/{file}")
            print(df.head(5))
            newFileName = 'train_file{}.csv.gz'.format(i)
            df.to_csv(
                
                f"{save_to_folder}/{newFileName}", 
                index=False, 
                compression="gzip"
            )
            i+=1

            print(f"Converted File in {round((time() - start)/60, 3)} minutes")
    


In [6]:
def draw_cv2(raw_strokes, size=size, lw=6):
    img = np.zeros((Base_Size, Base_Size))
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            img = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
    return (cv2.resize(img, (size, size))/ 255.)



#Data Augmentation
def image_generator(size, batchsize, ks, lw=6):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(data_Directory, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                x = np.zeros((len(df), size, size))
                df['drawing'] = [ast.literal_eval(pts) for pts in df['drawing'].values]   
                df['drawing'] = df['drawing'].apply(draw_cv2)
                x = np.vstack([a for a in df['drawing']]).reshape((len(df),size,size,1))
                y = tf.keras.utils.to_categorical(df.y, num_classes=num_class)
                yield x, y
                
                

def df_to_image_array(df, size=size, lw=6):
    df['drawing'] = [ast.literal_eval(pts) for pts in df['drawing'].values]
    x = np.zeros((len(df), size, size))
    print(df.shape)
    
    df['drawing'] = df['drawing'].apply(draw_cv2)
    x = np.vstack([a for a in df['drawing']]).reshape((len(df),size,size,1))
#     x = np.vstack([a for a in df['drawing']])
    return x

In [ ]:
from_folder = "../dataset/new/train_simplified"
data_Directory = "../dataset/train_simplified_csv_gz"
compress_csv_files(from_folder,data_Directory)

ih
0
----------------------------------------
Converting File: steak.csv
  countrycode                                            drawing  \
0          GB  [[[246, 241, 231, 206, 166, 157, 123, 106, 83,...   
1          HU  [[[236, 202, 113, 84, 58, 29, 16, 0, 0, 4, 17,...   
2          DE  [[[104, 71, 46, 23, 12, 2, 0, 14, 26, 44, 64, ...   
3          US  [[[3, 54, 79, 108, 161, 190, 213, 239, 247, 25...   
4          CA  [[[35, 15, 8, 5, 0, 8, 92, 121, 192, 213, 240,...   

             key_id  recognized                   timestamp   word  
0  6150633189015552        True  2017-03-02 17:40:48.349160  steak  
1  5597401035309056        True  2017-03-05 19:47:52.771110  steak  
2  6389337442746368        True  2017-01-23 14:29:47.792220  steak  
3  6706522417528832        True  2017-03-12 21:43:41.647620  steak  
4  5825039100280832        True  2017-03-07 22:27:33.663220  steak  
Converted File in 1.624 minutes
ih
1
----------------------------------------
Converting File: helicopte

Converted File in 1.034 minutes
ih
9
----------------------------------------
Converting File: bridge.csv
  countrycode                                            drawing  \
0          AT  [[[15, 39, 87, 178, 244], [126, 104, 83, 60, 5...   
1          CH  [[[131, 133], [12, 79]], [[130, 15, 56, 87, 56...   
2          DK  [[[35, 90, 137, 164], [119, 100, 100, 96]], [[...   
3          US  [[[0, 87, 110, 166, 255], [74, 42, 37, 54, 55]...   
4          US  [[[1, 9], [22, 112]], [[1, 28, 65], [18, 48, 1...   

             key_id  recognized                   timestamp    word  
0  5505601176076288        True  2017-03-09 15:23:32.521310  bridge  
1  6718277441028096       False  2017-03-02 17:36:59.776450  bridge  
2  6156662199025664        True  2017-03-28 19:25:22.017040  bridge  
3  6716403727990784        True  2017-03-07 02:55:49.706920  bridge  
4  5251071700107264        True  2017-03-24 23:13:25.484320  bridge  
Converted File in 1.128 minutes
ih
10
---------------------------

Converted File in 1.707 minutes
ih
18
----------------------------------------
Converting File: camouflage.csv
  countrycode                                            drawing  \
0          CA  [[[69, 82, 90, 93, 88, 76, 47, 45, 49, 69], [3...   
1          US  [[[74, 115, 158, 187, 205, 225, 230, 224, 208,...   
2          SE  [[[12, 1, 1, 15, 28, 36, 75, 182, 224, 249, 25...   
3          US  [[[0, 9, 23, 28, 31, 35, 64, 72, 76], [24, 29,...   
4          IN  [[[45, 77, 109, 68, 135, 94, 182, 144, 62, 141...   

             key_id  recognized                   timestamp        word  
0  5811310375731200        True  2017-01-27 02:41:07.490740  camouflage  
1  6384335143829504        True  2017-03-02 14:50:57.651190  camouflage  
2  4994950099894272        True  2017-03-07 08:20:33.881730  camouflage  
3  5013700735926272        True  2017-03-24 23:03:04.823120  camouflage  
4  5561131630329856        True  2017-03-25 15:56:24.467650  camouflage  
Converted File in 5.298 minutes
ih
1

Converted File in 1.954 minutes
ih
27
----------------------------------------
Converting File: pear.csv
  countrycode                                            drawing  \
0          US  [[[46, 21, 9, 0, 0, 8, 19, 46, 92, 127, 135, 1...   
1          US  [[[75, 74, 56, 38, 33, 34, 30, 18, 2, 0, 7, 16...   
2          US  [[[28, 17, 10, 0, 11, 11, 3, 3, 9, 22, 45, 69,...   
3          SA  [[[64, 80, 134, 172], [206, 164, 77, 49]], [[1...   
4          VN  [[[34, 26, 14, 2, 1, 14, 33, 49, 83, 101, 111,...   

             key_id  recognized                   timestamp  word  
0  5696968749744128        True  2017-01-29 02:40:55.895800  pear  
1  4914459074101248        True  2017-03-26 20:50:44.270270  pear  
2  4554473420619776        True  2017-03-07 20:23:43.873950  pear  
3  5024860285698048        True  2017-03-02 14:42:44.850870  pear  
4  6442724054007808        True  2017-01-31 05:04:10.537810  pear  
Converted File in 1.043 minutes
ih
28
----------------------------------------

Converted File in 1.968 minutes
ih
36
----------------------------------------
Converting File: alarm clock.csv
  countrycode                                            drawing  \
0          US  [[[111, 148, 161, 175, 199, 218, 231, 236, 234...   
1          US  [[[154, 144, 129, 86, 66, 45, 45, 50, 76, 111,...   
2          US  [[[26, 15, 1, 2, 12, 27, 50, 78, 111, 133, 163...   
3          US  [[[181, 174, 133, 110, 54, 35, 21, 12, 10, 18,...   
4          CZ  [[[120, 86, 46, 29, 24, 24, 27, 39, 72, 91, 11...   

             key_id  recognized                   timestamp         word  
0  5159910851477504        True  2017-03-21 13:02:16.246170  alarm clock  
1  4608088873107456        True  2017-03-01 21:42:04.745090  alarm clock  
2  5502112672776192        True  2017-03-28 17:52:35.433570  alarm clock  
3  6600682192764928        True  2017-03-20 18:40:57.954650  alarm clock  
4  6344944035823616        True  2017-03-26 18:40:59.233750  alarm clock  
Converted File in 2.725 minut

Converted File in 2.479 minutes
ih
45
----------------------------------------
Converting File: drums.csv
  countrycode                                            drawing  \
0          GB  [[[143, 131, 103, 58, 42, 36, 49, 69, 87, 125,...   
1          HK  [[[99, 51, 26, 15, 1, 0, 14, 37, 106, 133, 147...   
2          HK  [[[62, 118, 179, 229, 248, 255, 242, 225, 168,...   
3          US  [[[14, 15, 26, 38, 59, 62, 57, 56, 30, 20], [2...   
4          GB  [[[5, 2, 10, 78, 150, 152, 146, 145, 140, 3], ...   

             key_id  recognized                   timestamp   word  
0  6134195527090176        True  2017-03-16 02:19:30.826420  drums  
1  4587224731484160        True  2017-01-25 02:55:20.408780  drums  
2  5162628022272000        True  2017-03-14 03:38:06.546980  drums  
3  6360528580509696        True  2017-01-31 17:40:42.328700  drums  
4  6088366162444288       False  2017-03-11 16:47:37.175380  drums  
Converted File in 2.08 minutes
ih
46
----------------------------------

Converted File in 4.982 minutes
ih
54
----------------------------------------
Converting File: hexagon.csv
  countrycode                                            drawing  \
0          US  [[[93, 38, 0], [8, 63, 96]], [[13, 19], [108, ...   
1          US  [[[65, 62, 4, 0, 24, 101], [2, 10, 78, 89, 114...   
2          BR  [[[0, 76, 93, 192, 200, 255, 246, 165, 166, 15...   
3          US  [[[124, 113, 102, 22, 0, 51, 68, 126, 148, 156...   
4          US  [[[58, 29, 2, 1, 11, 35, 121, 151, 183, 224, 2...   

             key_id  recognized                   timestamp     word  
0  6695476885716992        True  2017-03-17 20:20:25.234330  hexagon  
1  4509550780612608        True  2017-03-04 14:54:41.166690  hexagon  
2  5401676863242240        True  2017-01-30 13:19:15.170880  hexagon  
3  5388880746381312        True  2017-01-27 17:19:10.248280  hexagon  
4  5724753627185152        True  2017-03-05 02:59:37.331420  hexagon  
Converted File in 0.798 minutes
ih
55
-------------------

In [15]:
# generating training data for 1 csv file :- (for testing functioning) 
# valid_df = pd.read_csv('./dataset/test_simplified.csv')
# valid_df = pd.read_csv(os.path.join(data_Directory, 'train_file{}.csv.gz'.format(num_csv - 1)), nrows=30000)
# valid_df.head()
# x_valid = df_to_image_array(valid_df, size)
# y_valid = tf.keras.utils.to_categorical(valid_df.y, num_classes=num_class)
# print(x_valid.shape, y_valid.shape)

In [40]:
# generating training data
# train_datagen = image_generator(size=size, batchsize=batchsize, ks=range(num_csv - 1))

In [18]:
i=0
print('train_file{}.csv.gz'.format(i))


train_file0.csv.gz
